### Setup

In [1]:
import sys
import os

# Add the project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
from src.idspy.core.pipeline import PipelineEvent, ObservablePipeline
from src.idspy.core.state import State
from src.idspy.core.step import Step
from src.idspy.events.bus import EventBus
from src.idspy.events.events import Event, only_id

ImportError: cannot import name 'only_id' from 'src.idspy.events' (/Users/simonealbero/Documents/01_work/03_code/idspy/src/idspy/events/__init__.py)

### Create an `EventBus` and register subscribers

In [ ]:
bus = EventBus()


# All events: minimal one-line log
@bus.on(EventBus.ALL)
def log_all(ev: Event) -> None:
    print(f"[ALL] {ev.type} :: {ev.id}")


# Before/After step: show index + requires/provides
@bus.on(PipelineEvent.BEFORE_STEP.value)
def on_before(ev: Event) -> None:
    idx = ev.payload.get("index")
    req = ev.payload.get("requires", [])
    prov = ev.payload.get("provides", [])
    print(f"[BEFORE] idx={idx} step={ev.id} requires={req} provides={prov}")


@bus.on(PipelineEvent.BEFORE_STEP.value, only_id("Demo.Sum"))
def on_before_sum(ev: Event) -> None:
    print(f"[BEFORE] summing step={ev.id}")


@bus.on(PipelineEvent.AFTER_STEP.value)
def on_after(ev: Event) -> None:
    idx = ev.payload.get("index")
    print(f"[AFTER]  idx={idx} step={ev.id}")


# Error handler
@bus.on(PipelineEvent.ON_ERROR.value)
def on_err(ev: Event) -> None:
    print(f"[ERROR] step={ev.id} :: {ev.payload.get('error')}")

### Define a couple of simple `Steps`

In [ ]:
class Load(Step):
    def __init__(self):
        super().__init__(provides=["data"])

    def run(self, state: State) -> None:
        state["data"] = [1, 2, 3]


class Sum(Step):
    def __init__(self):
        super().__init__(requires=["data"], provides=["sum"])

    def run(self, state: State) -> None:
        state["sum"] = sum(state["data"])


class Boom(Step):
    def __init__(self):
        super().__init__(requires=["missing"])

    def run(self, state: State) -> None:
        # never reached because requires isn't satisfied
        pass


### Build and run an `ObservablePipeline`

In [ ]:
p = ObservablePipeline([Load(), Sum()], name="Demo", bus=bus)

s = State()
p(s)
print("STATE:", s.to_dict())
# Expected:
# [ALL] PipelineEvent.PIPELINE_START :: Demo
# [BEFORE] idx=0 step=Demo.Load requires=[] provides=['data']
# [ALL] PipelineEvent.BEFORE_STEP :: Demo.Load
# [AFTER]  idx=0 step=Demo.Load
# [ALL] PipelineEvent.AFTER_STEP :: Demo.Load
# [BEFORE] idx=1 step=Demo.Sum requires=['data'] provides=['sum']
# [BEFORE] summing step=Demo.Sum
# [ALL] PipelineEvent.BEFORE_STEP :: Demo.Sum
# [AFTER]  idx=1 step=Demo.Sum
# [ALL] PipelineEvent.AFTER_STEP :: Demo.Sum
# [ALL] PipelineEvent.PIPELINE_END :: Demo
# STATE: {'data': [1, 2, 3], 'sum': 6}

[ALL] PipelineEvent.PIPELINE_START :: Demo
[BEFORE] idx=0 step=Demo.Load requires=[] provides=['data']
[ALL] PipelineEvent.BEFORE_STEP :: Demo.Load
[AFTER]  idx=0 step=Demo.Load
[ALL] PipelineEvent.AFTER_STEP :: Demo.Load
[BEFORE] idx=1 step=Demo.Sum requires=['data'] provides=['sum']
[BEFORE] summing step=Demo.Sum
[ALL] PipelineEvent.BEFORE_STEP :: Demo.Sum
[AFTER]  idx=1 step=Demo.Sum
[ALL] PipelineEvent.AFTER_STEP :: Demo.Sum
[ALL] PipelineEvent.PIPELINE_END :: Demo
STATE: {'data': [1, 2, 3], 'sum': 6}


### Error path (triggers on_error)

In [ ]:
p_err = ObservablePipeline([Boom()], name="ErrDemo", bus=bus)

try:
    p_err(State())
except KeyError:
    pass
# Expected:
# [ALL] PipelineEvent.PIPELINE_START :: ErrDemo
# [BEFORE] idx=0 step=ErrDemo.Boom requires=['missing'] provides=[]
# [ALL] PipelineEvent.BEFORE_STEP :: ErrDemo.Boom
# [ERROR] step=ErrDemo.Boom :: KeyError("Boom: missing ['missing']")
# [ALL] PipelineEvent.ON_ERROR :: ErrDemo.Boom
# [ALL] PipelineEvent.PIPELINE_END :: ErrDemo

[ALL] PipelineEvent.PIPELINE_START :: ErrDemo
[BEFORE] idx=0 step=ErrDemo.Boom requires=['missing'] provides=[]
[ALL] PipelineEvent.BEFORE_STEP :: ErrDemo.Boom
[ERROR] step=ErrDemo.Boom :: KeyError("Boom: missing ['missing']")
[ALL] PipelineEvent.ON_ERROR :: ErrDemo.Boom
[ALL] PipelineEvent.PIPELINE_END :: ErrDemo
